In [2]:
import os
import torch
import torch.nn as nn 
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224,224)),   # resizing because most models are made for inputs 2244,224
    transforms.ToTensor(),  # for normalixing the image /255
    transforms.Normalize(mean=[0.5,0.5,0.5],
                         std=[0.5,0.5,0.5])   # to  get a range between -1 to 1 do that gradients are in a sig zag movement

])



class RealAiDAtaset(torch.utils.data.Dataset):

    def __init__(self,root_dir,transform=None):
        
        self.image_path = []
        self.labels = []
        self.transform = transform

        for label,folder in enumerate(['REAL','FAKE']):
            folder_path = os.path.join(root_dir,folder)

            for img_file in (os.listdir(folder_path)):
                self.image_path.append(os.path.join(folder_path,img_file))
                self.labels.append(label)


    def __len__(self):
        return len(self.image_path)
                

    def __getitem__(self,idx):
        image = Image.open(self.image_path[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image,label
    

Real = RealAiDAtaset(root_dir=r"E:\pytorch\dataset\train", transform=transform)    # testing the class by passing values if the images are drawn corectly



train_loader = DataLoader(
    Real,
    batch_size=16,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)





class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()


        self.conv_layers = nn.Sequential(

            nn.Conv2d(3,32,kernel_size=3,padding=1),   # 1st layer
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32,64,kernel_size=3,padding=1),   #2nd layer
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64,128,kernel_size=3,padding=1),   #3rd layer
            nn.ReLU(),
            nn.MaxPool2d(2)

        )


        self.fv_layer = nn.Sequential(

            nn.Flatten(),
            nn.Linear(128*28*28,512),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,1)   # inpopu tvalue 512 and ouput value 1
        )


    def forward(self,x):

        x = self.conv_layers(x)
        x = self.fv_layer(x)    # input = (batchsize,features)

        return x



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # to set the device


model  = SimpleCNN().to(device)

criterion = nn.BCEWithLogitsLoss()


optimizer = optim.Adam(model.parameters(),lr=0.001)


num_epochs = 5

for epoch in range(num_epochs):

    model.train()
    running_loss = 0.0

    for imgs,labels in train_loader:

        imgs = imgs.to(device)

        labels = labels.float().to(device)

        optimizer.zero_grad()   # to  remove the prevous batcg gradientts


        outputs = model(imgs).squeeze()   # because the BCElo...expects (batchsize,)   output

        loss = criterion(outputs,labels)

        loss.backward()

        optimizer.step()


        running_loss += loss.item()
        print("Using device:", device)


    avg_loss = running_loss / len(train_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda


KeyboardInterrupt: 

In [ ]:
test_dataset = RealAiDAtaset(
    root_dir="/content/dataset/test",  # path to your test images
    transform=transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False  # don't shuffle test data
)


model = SimpleCNN().to(device)
model.load_state_dict(torch.load("/content/dataset/simple_cnn.pth"))
model.eval()  # important for evaluation (disables dropout)

all_preds = []
all_labels = []

with torch.no_grad():  # disables gradient computation
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device).float()

        outputs = model(imgs).squeeze()  # (batch_size,)
        preds = torch.sigmoid(outputs)  # convert logits to probabilities
        preds = (preds > 0.5).float()   # threshold to get class 0 or 1

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy = accuracy_score(all_labels, all_preds)
print("Test Accuracy:", accuracy)

print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
print("Classification Report:\n", classification_report(all_labels, all_preds))


In [ ]:
test_dataset = RealAiDAtaset(root_dir="/content/dataset/test",transform=transform)

test_loader = DataLoader(test_dataset,batch_size=16,shuffle=False)

model = SimpleCNN().to(device)
model.load_state_dict(torch.load("E:\pytorch\simple_cnn.pth"))
model.eval()


with torch.no_grad():
    for imgs,labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device).float()
        
        outputs = model(imgs).squeeze()
        preds = torch.sigmoid(outputs)

        all_preds.extend(preds.cpu().numpy())

        all_labels.extend(labels.cpu().numpy())


from sklearn.metrics import accuracy_score ,confusion_matrix,classofication_report

accuracy = accuracy_score(all_labels,all_preds)

print("accuracy",accuracy_score(all_labels,all_preds))
print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
print("Classification Report:\n", classification_report(all_labels, all_preds))


